In [2]:
import mfem.ser as mfem
from mfem.common.arg_parser import ArgParser
import numpy as np
from glvis import glvis
import matplotlib.pyplot as plt
import mpi4py.MPI as MPI
#print(MPI.Get_version())
import math
import h5py
import os

In [ ]:
def create_2D_mesh(nx, ny, x_start, x_end, y_start, y_end):
    """
    Creates a 2D mesh with the specified intervals and coordinate ranges.

    Args:
        nx (int): Number of intervals in the x-direction.
        ny (int): Number of intervals in the y-direction.
        x_start (float): Starting x-coordinate.
        x_end (float): Ending x-coordinate.
        y_start (float): Starting y-coordinate.
        y_end (float): Ending y-coordinate.

    Returns:
        mesh: The updated mesh with vertex coordinates set accordingly.
    """
    # Generate equally spaced coordinates for x and y
    x_coords = np.linspace(x_start, x_end, nx + 1)
    y_coords = np.linspace(y_start, y_end, ny + 1)
    
    # Create the mesh with initial x and y values set to 0 (they will be updated)
    mesh = mfem.Mesh(nx, ny, "QUADRILATERAL", True, 0.0, 0.0)
    
    # Retrieve the vertex array from the mesh
    verts = mesh.GetVertexArray()
    
    # Check if the number of vertices is as expected: (nx+1) * (ny+1)
    expected_num = (nx + 1) * (ny + 1)
    num_verts = mesh.GetNV()
    
    if num_verts != expected_num:
        print("Warning: Unexpected number of vertices! ({} != {})".format(num_verts, expected_num))
    
    # Update the vertex coordinates; vertices are stored in row-major order
    k = 0
    for j in range(ny + 1):
        for i in range(nx + 1):
            verts[k][0] = x_coords[i]
            verts[k][1] = y_coords[j]
            k += 1
    #mesh.Print("output.mesh")

    return mesh

def gauss_legendre_dirs(N_dir):
    """
    Compute Gauss-Legendre quadrature points (mu_i) and weights (w_i)
    for discrete ordinates. Typically used for [-1,1].
    Returns (mu[], w[]) each of length N_dir.
    """
    mu = []
    w  = []

    gg_mu, gg_w = np.polynomial.legendre.leggauss(N_dir)
    
    mu = gg_mu.tolist()
    w  = gg_w.tolist()
    return mu, w

def read_data(nE):
    """
    Read cross-section and stopping power data from an HDF5 file.

    Expects datasets "xs_t_{nE}", "xs_s_{nE}", and "S_{nE}".

    Args:
        nE (int): Number of energy groups.

    Returns:
        tuple: (xs_t_arr, xs_s_arr, S_arr) as numpy arrays.
    """
    data_file = "data.h5"

    with h5py.File(data_file, "r") as f:
        E_arr = f[f"E_{nE}"][:]
        E_grid_arr = f[f"E_{nE}"][:]
        xs_t_arr = f[f"xs_t_{nE}"][:]
        xs_s_arr = f[f"xs_s_{nE}"][:]
        S_arr    = f[f"S_{nE}"][:]
        
    return E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr

class TotalXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the total cross-section Σ_t(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    cross-section value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, xs_t_data, E_start, E_end):
        super(TotalXSCoefficient, self).__init__()
        self.xs_t_data = xs_t_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1] 
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_t_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_t_data[group])
    
class ScatteringXSCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the scattering cross-section Σ_s(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    scattering cross-section value. Here, y=0 corresponds to E = E_start and y=1 
    corresponds to E = E_end.
    """
    def __init__(self, xs_s_data, E_start, E_end):
        super(ScatteringXSCoefficient, self).__init__()
        self.xs_s_data = xs_s_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.xs_s_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.xs_s_data[group])

class StoppingPowerCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the stopping power S(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    S(E) value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficient, self).__init__()
        self.S_data = S_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.S_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.S_data[group])
    
class StoppingPowerDerivativeCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the derivative of the stopping power S(E), i.e., S'(E).

    Maps the normalized energy coordinate (x[1]) in [0,1] to the corresponding 
    derivative value. Here, y=0 corresponds to E = E_start and y=1 corresponds to 
    E = E_end.
    """
    def __init__(self, dS_data, E_start, E_end):
        super(StoppingPowerDerivativeCoefficient, self).__init__()
        self.dS_data = dS_data
        self.E_start = E_start
        self.E_end = E_end

    def EvalValue(self, x):
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.dS_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))
        return float(self.dS_data[group])
    
class InflowCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the inflow boundary condition.
    Returns a prescribed inflow flux value.
    """
    def __init__(self, inflow_value):
        super(InflowCoefficient, self).__init__()
        self.inflow_value = inflow_value

    def EvalValue(self, x):
        return self.inflow_value
    

class QCoefficient(mfem.PyCoefficient):
    """
    Coefficient for the source term Q(x,E).

    Maps the normalized energy coordinate (x[1] in [0,1]) to the corresponding 
    Q value. Here, y=0 corresponds to E = E_start and y=1 corresponds to E = E_end.
    
    If Q_data is given as the scalar 0, then the coefficient always returns 0,
    and E_start and E_end are not required.
    """
    def __init__(self, Q_data, E_start=None, E_end=None):
        super(QCoefficient, self).__init__()

        if isinstance(Q_data, (int, float)) and Q_data == 0:
            self.scalar_zero = True
            self.Q_data = None

        else:
            if E_start is None or E_end is None:
                raise ValueError("For non-zero Q_data, E_start and E_end must be provided.")
            self.scalar_zero = False
            if isinstance(Q_data, (int, float)):

                self.Q_data = [Q_data]
            else:
                self.Q_data = Q_data
            self.E_start = E_start
            self.E_end = E_end

    def EvalValue(self, x):
        if self.scalar_zero:
            return 0.0
        
        y = x[1]
        E = self.E_start + y * (self.E_end - self.E_start)
        n_groups = len(self.Q_data)
        group = min(n_groups - 1, int((E - self.E_start) / (self.E_end - self.E_start) * n_groups))

        return float(self.Q_data[group])

def compute_S_derivative(E, S):
    """
    Compute the derivative dS/dE using finite differences.
    
    Args:
        E (ndarray): 1D array of energy values.
        S (ndarray): 1D array of S values corresponding to the energies.
    
    Returns:
        dS_dE (ndarray): Array of the finite difference approximation of the derivative dS/dE.
    """
    n = len(S)
    dS_dE = np.zeros(n)
    
    # Forward difference for the first point
    dS_dE[0] = (S[1] - S[0]) / (E[1] - E[0])
    
    # Central difference for interior points
    for i in range(1, n - 1):
        dS_dE[i] = (S[i + 1] - S[i - 1]) / (E[i + 1] - E[i - 1])
    
    # Backward difference for the last point
    dS_dE[-1] = (S[-1] - S[-2]) / (E[-1] - E[-2])
    
    return dS_dE

class MeshTransformer2D:
    """
    Transforms a 2D mesh defined on the unit square [0,1]^2 into the domain 
    [x_start, x_end] x [E_start, E_end]. The first coordinate (x) is scaled by L, and the 
    second coordinate (interpreted as a normalized energy variable) is mapped 
    linearly from [0,1] to [E_start, E_end].
    """
    def __init__(self, x_start, x_end, E_start, E_end):
        self.L = x_end - x_start
        self.E_start = E_start
        self.E_end = E_end

    def Transform(self, mesh):
        """
        Transforms the vertices of the given mesh.

        Parameters:
          mesh : an mfem.Mesh instance defined on [0,1]^2.
        """
        # Get the vertex array (returned as a tuple) and convert it to a NumPy array.
        verts = np.array(mesh.GetVertexArray())
        
        # Assuming the mesh is 2D:
        # - Multiply the x-coordinate (column 0) by L.
        # - Map the second coordinate (energy) from [0,1] to [E_start, E_end].
        verts[:, 0] *= self.L
        verts[:, 1] = self.E_start + verts[:, 1] * (self.E_end - self.E_start)
        
        # Flatten the 2D vertex array into a 1D list (row-major order).
        flat_verts = verts.flatten().tolist()
        
        # Create an mfem.Vector from the flattened list.
        v_vec = mfem.Vector(flat_verts)
        
        # Update the mesh vertices.
        mesh.SetVertices(v_vec)
        return mesh

import mfem.ser as mfem

class VelocityCoefficient(mfem.VectorPyCoefficientBase):
    """
    Coefficient for the velocity vector in the transport equation.
    
    This coefficient defines:
         v(x) = [ μ, S(E) ]
    where:
       - μ is a provided scalar (e.g., the discrete ordinate value),
       - S(E) is computed by S_coef.Eval(x) using the energy coordinate x[1].
    
    The class inherits from VectorPyCoefficientBase (the Python wrapper for the
    C++ VectorPyCoefficientBase) and overrides the _EvalPy method.
    """
    def __init__(self, mu, S_coef):
        # Call the base class constructor with dimension 2 and tdep = 0 (i.e. not time-dependent).
        mfem.VectorPyCoefficientBase.__init__(self, 2, 0)
        self.mu = mu
        self.S_coef = S_coef

    def _EvalPy(self, V, ip):
        """
        Evaluate the velocity at the integration point.
        
        Parameters:
          V : mfem.Vector that should be set with the coefficient value.
          ip: An IntegrationPoint (which has a member 'x' for the coordinates).
        """
        # Extract the coordinate x from the integration point.
        x = ip.x
        # Set the first component to mu and the second component to S(E)
        V.Set(0, self.mu)
        V.Set(1, self.S_coef.Eval(x))





NameError: name 'mfem' is not defined

In [6]:
# This will be the init function:
nx = 20
nE = 50
x_start = 0.0
x_end = 0.3
E_start = 1.0
E_end = 0.01
N_ang = 8
order = 1


#mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
#dim = mesh.Dimension()

#fec = mfem.DG_FECollection(order, dim)
#fes = mfem.FiniteElementSpace(mesh, fec)

#ndofs = fes.GetVSize()
#print(f"Nubmer of dimensions: {dim}")
#print(f"Number of finite element unknowns: {ndofs}")

#a = mfem.BilinearForm(fes)

#ess_tdof_list = mfem.intArray()
#dir_bdr = mfem.intArray(mesh.bdr_attributes.Max())
#dir_bdr.Assign(1)
#dir_bdr.Size()

#print(f"Number of finite element dir_bdr.Size(): {dir_bdr.Size()}")
#print(f"Number of finite element ess_tdof_list.Size(): {ess_tdof_list.Size()}")

inflow_coeff = InflowCoefficient(1.0)
E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(50)
dS_dE_arr = compute_S_derivative(E_arr, S_arr)
xs_t_coeff = TotalXSCoefficient(xs_t_arr, E_start, E_end)
xs_s_coeff = ScatteringXSCoefficient(xs_s_arr, E_start, E_end)
S_coeff = StoppingPowerCoefficient(S_arr, E_start, E_end)
dS_dE_coeff = StoppingPowerDerivativeCoefficient(dS_dE_arr, E_start, E_end)


mu_vals, w_vals = gauss_legendre_dirs(N_ang)
psi_mu_list = []

for mu, w in zip(mu_vals, w_vals):
    print("Solving for μ =", mu)
    # Create a 2D mesh over [0,L] x [E_min,E_max] using quadrilaterals.
    mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
    transformer = MeshTransformer2D(x_start, x_end, E_start, E_end)
    transformer.Transform(mesh)
    dim = mesh.Dimension()

    # Define the DG finite element space using L2 elements
    fec = mfem.L2_FECollection(order, dim)
    fes = mfem.FiniteElementSpace(mesh, fec)
    print("Number of unknowns:", fes.GetTrueVSize())
    v_coef = VelocityCoefficient(mu, S_coeff)

    # Set up the bilinear form a(.,.) for the DG formulation.
    a = mfem.BilinearForm(fes)
    # Volume integrator for convection term: μ ∂ψ/∂x + S(E) ∂ψ/∂E.
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coef))
    # Mass integrators for the reaction term: [S'(E) + Σₜ(E)] ψ.
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, -1.0))
    a.AddBdrFaceIntegrator(mfem.NonconservativeDGTraceIntegrator(v_coef, -1.0))
    a.Assemble()
    a.Finalize()
    










Solving for μ = -0.9602898564975362
Number of unknowns: 4000


AttributeError: 'Vector' object has no attribute 'x'

In [14]:
'''b = mfem.LinearForm(fes)
q_coef = QCoefficient(0)
b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coef))
b.Assemble()'''

'b = mfem.LinearForm(fes)\nq_coef = QCoefficient(0)\nb.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coef))\nb.Assemble()'

In [5]:
'''

for lev in range(ref_levels):
    mesh.UniformRefinement()
    if mesh.NURBSext:
        mesh.SetCurvature(max(order, 1))
    bb_min, bb_max = mesh.GetBoundingBox(max(order, 1))


    
# Example usage:
if __name__ == '__main__':
    # Generate 51 energy values from 1.0 to 0.01 (inclusive)

    # Define S as an example function of E, e.g., S(E) = 0.5 * E + 0.1
    E, E_grid_arr, xs_t_arr, xs_s_arr, S = read_data(50)
    # Compute the derivative dS/dE
    dS_dE = compute_derivative(E, S)
    
    # Print the results
    print("Energy values (E):")
    print(E)
    print("\nS values:")
    print(S)
    print("\ndS/dE values:")
    print(dS_dE)
    '''

'\n\nfor lev in range(ref_levels):\n    mesh.UniformRefinement()\n    if mesh.NURBSext:\n        mesh.SetCurvature(max(order, 1))\n    bb_min, bb_max = mesh.GetBoundingBox(max(order, 1))\n\n\n    \n# Example usage:\nif __name__ == \'__main__\':\n    # Generate 51 energy values from 1.0 to 0.01 (inclusive)\n\n    # Define S as an example function of E, e.g., S(E) = 0.5 * E + 0.1\n    E, E_grid_arr, xs_t_arr, xs_s_arr, S = read_data(50)\n    # Compute the derivative dS/dE\n    dS_dE = compute_derivative(E, S)\n    \n    # Print the results\n    print("Energy values (E):")\n    print(E)\n    print("\nS values:")\n    print(S)\n    print("\ndS/dE values:")\n    print(dS_dE)\n    '

In [ ]:
        # Define the linear form (right-hand side): inflow boundary condition
        b = mfem.LinearForm(fes)
        # Inflow is applied at (x=x_start, E=E_start); elsewhere (e.g., x=x_end for μ<0) it is vacuum (0)
        inflow_2d = Inflow2DCoefficient(x_start, E_start)
        b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_2d, v_coef, -1.0))
        b.Assemble()

        # Solve the linear system using GMRES
        A = a.SpMat()
        prec = mfem.GSSmoother(A)
        solver = mfem.GMRESSolver()
        solver.SetOperator(A)
        solver.SetPreconditioner(prec)
        solver.SetRelTol(1e-12)
        solver.SetAbsTol(1e-12)
        solver.SetMaxIter(500)
        solver.SetKDim(30)
        solver.SetPrintLevel(1)

        # Initialize the solution GridFunction
        psi = mfem.GridFunction(fes)
        psi.Assign(0.0)
        solver.Mult(b, psi)
        print("GMRES iterations (μ = {}):".format(mu), solver.GetNumIterations(), " final norm:", solver.GetFinalNorm())

        # Save the solution for this angular direction and store in list
        psi_mu_list.append((mu, psi))
        out_name = f"psi_mu_{mu:.3f}.gf"
        psi.Save(out_name)
        print(f"Solution for μ = {mu:.3f} saved to {out_name}.\n")

    # Angular integration: compute total solution psi_total(x,E) = Σ_{μ} w(μ) ψ(x,E,μ)
    # We assume all angular solutions are defined on the same FE space.
    psi_total = mfem.GridFunction(fes)
    psi_total.Assign(0.0)
    for mu, psi in psi_mu_list:
        weight = w_vals[mu_vals.index(mu)]
        psi_total.Add(weight, psi)
    out_total = "psi_total.gf"
    psi_total.Save(out_total)
    print("Angle-integrated solution saved to", out_total)

    # Save the mesh for GLVis visualization
    mesh.Print("mesh.mesh")
    print("Mesh saved to mesh.mesh")

    #---------------------------------------------------
    # GLVis Visualization
    #---------------------------------------------------
    # Visualize the total (angle-integrated) solution with GLVis.
    # The title is set to "Total Angular Flux" and we wait for 500 ms.
    glvis.draw_solution(mesh, psi_total, "Total Angular Flux", 500)

    #---------------------------------------------------
    # Dose Calculation and Plotting (Matplotlib)
    #---------------------------------------------------
    # Here, we compute the average flux in the energy (y) direction for each x cell,
    # then define dose as the normalized flux relative to the value at x=0.
    x_coords = []
    avg_flux = []
    for i in range(nx):
        # x coordinate of the cell center
        x_val = x_start + (x_end - x_start)*(i + 0.5)/nx
        cell_vals = []
        for j in range(ny):
            # y (energy) coordinate of the cell center
            y_val = E_start + (E_end - E_start)*(j + 0.5)/ny
            # Compute cell index in row-major order
            cell_index = i * ny + j
            cell_vals.append(psi_total[cell_index])
        x_coords.append(x_val)
        avg_flux.append(np.mean(cell_vals))
    avg_flux = np.array(avg_flux)
    # Define dose as the normalized average flux (dose at x divided by dose at x=0)
    dose = avg_flux / avg_flux[0]

    # Plot the average flux along x
    plt.figure()
    plt.plot(x_coords, avg_flux, 'o-', label='Average Flux')
    plt.xlabel("x (cm)")
    plt.ylabel("Average Angular Flux ψ(x)")
    plt.title("Average Angular Flux vs. x")
    plt.grid(True)
    plt.legend()

    # Plot the computed dose profile
    plt.figure()
    plt.plot(x_coords, dose, 's-', color='red', label='Dose (Normalized)')
    plt.xlabel("x (cm)")
    plt.ylabel("Dose (normalized)")
    plt.title("Dose Profile vs. x")
    plt.grid(True)
    plt.legend()
    plt.show()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 34)